# Set up

In [ ]:
project_folder = "/content/drive/MyDrive/plant_disease_predictor_flexible"

from google.colab import drive
drive.mount('/content/drive')

%cd "{project_folder}"
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/water_quality_predictor
Already up to date.


# Cài đặt môi trường

In [ ]:
!git pull
!pip install -r requirements.txt

Already up to date.
Obtaining file:///content/drive/MyDrive/water_quality_predictor (from -r requirements.txt (line 17))
  Preparing metadata (setup.py) ... done
  Attempting uninstall: classifier
    Found existing installation: classifier 0.0.0
    Uninstalling classifier-0.0.0:
      Successfully uninstalled classifier-0.0.0
  Running setup.py develop for classifier


# Thư viện

In [ ]:
from Mylib import myfuncs
from src.model_training import mt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Flatten,
    Dense,
)
from tensorflow.keras.optimizers import RMSprop

# Các tham số

In [ ]:
image_size = 224
channels = 3
scoring = 'accuracy'
target_score = 0.3

model_name = "model1"
epochs = 10 
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, min_delta=1)
]

models = [
    (1, 
        [
            Flatten(), 
            Dense(units = 64, activation = 'relu'), 
            Dense(units = 3, activation = 'softmax'), 
        ]
    ), 
    (2, 
        [
            Flatten(), 
            Dense(units = 64, activation = 'relu'), 
            Dense(units = 64, activation = 'relu'), 
            Dense(units = 64, activation = 'relu'), 
            Dense(units = 3, activation = 'softmax'), 
        ]
    ), 
    (3, 
        [
            Flatten(), 
            Dense(units = 64, activation = 'relu'), 
            Dense(units = 64, activation = 'relu'), 
            Dense(units = 3, activation = 'softmax'), 
        ]
    ), 
    
]
optimizer = RMSprop()
loss = "sparse_categorical_crossentropy"

# Run 

In [ ]:
# Các tham số 
train_ds_path = "artifacts/data_ingestion/train_ds"
val_ds_path = "artifacts/data_ingestion/val_ds"
model_training_path = f"artifacts/model_training/{model_name}"
model_indices, models = zip(*models)
plot_dir = "artifacts/plot/components"
model_checkpoint_monitor = f"val_{scoring}"
metrics = [
    scoring
]

# Tạo các thư mục 
myfuncs.create_directories([model_training_path, plot_dir])

# Load dữ liệu 
train_ds, val_ds = mt.load_train_ds_and_val_ds(train_ds_path, val_ds_path)
num_models = len(models)

# Create các models và save trước khi training
mt.create_and_save_models_before_training(
    model_training_path,
    model_indices,
    models,
    image_size,
    channels,
)

# Train các models và save models 
mt.train_and_save_models(
    model_training_path,
    model_indices,
    num_models,
    train_ds,
    val_ds,
    epochs,
    callbacks,
    model_name,
    target_score,
    model_checkpoint_monitor,
    scoring,
    plot_dir,
    optimizer,
    loss,
    metrics,
)